In [33]:
import pandas as pd
# 여러 변수 출력 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [34]:
df_bus = pd.read_csv('bus_station_coordinate.csv', encoding='cp949')

In [35]:
df_sub = pd.read_csv('subway_station_coordinate.csv',index_col=0)

In [36]:
df_bus['정류장_유형'].unique()
len(df_bus)
df_sub.head()

array(['가상정류장', '마을버스', '일반차로', '일반중앙차로', '중앙차로', '가로변전일', nan, '가로변시간'],
      dtype=object)

69942

,line,name,lat,lng
0,01호선,가산디지털단지,37.481072,126.882343
1,01호선,간석,37.464737,126.694181
2,01호선,개봉,37.494594,126.858680
3,01호선,관악,37.419232,126.908706
4,01호선,광명,37.416182,126.884466


In [37]:
df_bus = df_bus[df_bus['정류장_유형']!='가상정류장']
df_bus = df_bus[df_bus['정류장_유형']!='마을버스']
len(df_bus)

42044

In [38]:
# !pip install haversine

In [39]:
from haversine import haversine

In [43]:
df_bus_one = df_bus.iloc[0]
bus_spot = (df_bus_one['경도'], df_bus_one['위도'])
bus_spot

df_sub['distance'] = 0
for i in range(len(df_sub)):
    df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i]))



(37.319934, 126.710485)

/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_7589/2574969012.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['distance'][i] = haversine(bus_spot, (df_sub['lat'][i], df_sub['lng'][i]))


In [44]:
near_sub = df_sub.loc[df_sub['distance'].idxmin()]
near = pd.concat([df_bus_one, near_sub])
near_sub
near

line              04호선
name                정왕
lat          37.351735
lng         126.742989
distance       4.55656
Name: 199, dtype: object

정류장_ID              500000286
정류장_명칭                  아남중전기
정류장_유형                   일반차로
정류장_번호                25963.0
위도                 126.710485
경도                  37.319934
버스도착정보안내기_설치_여부         미설치  
line                     04호선
name                       정왕
lat                 37.351735
lng                126.742989
distance              4.55656
dtype: object

In [45]:
new_df = pd.DataFrame(columns=near.keys())
new_df

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부,line,name,lat,lng,distance


In [46]:
# !pip install tqdm

In [47]:
import time
from tqdm import tqdm

In [60]:
len(df_bus)

42044

In [64]:
# len(df_bus) 42044
for j in tqdm(range(40000,42044)):

    df_bus_one = df_bus.iloc[j]
    bus_spot = (df_bus_one['경도'], df_bus_one['위도']) #버스 지점 하나

    #거리계산 시작
    df_sub['distance'] = 0 #거리 열 추가
    for i in range(len(df_sub)):
        sub_spot = (df_sub['lat'][i], df_sub['lng'][i])
        df_sub['distance'][i] = haversine(bus_spot, sub_spot) #모든 구간 거리계산 완료

    try:
        #그 중에 가장 가까운 행(내림차순으로 가장 첫번째 행)
        near_sub = df_sub.iloc[df_sub['distance'].idxmin()]
        
    except:
        near_sub = pd.Series({"line":'','name':'','code':'','lat':'','lng':'','distance':''})

    near = pd.concat([df_bus_one, near_sub]) #버스 정보 + 가장 가까운 지하철역 정보

    #비어있던 new_df에 한줄씩 추가
    new_df.loc[j] = near

  0%|          | 0/2044 [00:00<?, ?it/s]/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_7589/1912373981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['distance'][i] = haversine(bus_spot, sub_spot) #모든 구간 거리계산 완료
/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_7589/1912373981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['distance'][i] = haversine(bus_spot, sub_spot) #모든 구간 거리계산 완료
/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_7589/1912373981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

In [51]:
# line              01호선
# name                역곡
# code            1803.0
# lat          37.485178
# lng         126.811502
# distance           3.0
# Name: 49, dtype: object

near_sub = pd.Series({"line":'','name':'','code':'','lat':'','lng':'','distance':''})
near_sub

line        
name        
code        
lat         
lng         
distance    
dtype: object

In [67]:
len(new_df)
len(new_df['정류장_ID'].unique())

42044

42044

In [ ]:
near = pd.Series({""})

In [68]:
new_df.to_csv('거리계산완료.csv')

In [69]:
new_df.keys()

Index(['정류장_ID', '정류장_명칭', '정류장_유형', '정류장_번호', '위도', '경도', '버스도착정보안내기_설치_여부',
       'line', 'name', 'lat', 'lng', 'distance'],
      dtype='object')